In [41]:
import pandas as pd
import os

# ---------------------- CAMINHOS ----------------------

path_apps      = r'C:\Users\giuliasilva\Desktop\Estudo\POS\TC - Modulo 05\application_web\original_data\applicants.json'
path_vagas     = r'C:\Users\giuliasilva\Desktop\Estudo\POS\TC - Modulo 05\application_web\original_data\vagas.json'
path_prospects = r'C:\Users\giuliasilva\Desktop\Estudo\POS\TC - Modulo 05\application_web\original_data\prospects.json'

destino = r'C:\Users\giuliasilva\Desktop\Estudo\POS\TC - Modulo 05\application_web\data'
os.makedirs(destino, exist_ok=True)

# ---------------------- APLICANTS ----------------------

df_apps_raw = pd.read_json(path_apps, orient='index')

infos_basicas               = pd.json_normalize(df_apps_raw['infos_basicas']).add_prefix('infos_basicas_')
informacoes_pessoais        = pd.json_normalize(df_apps_raw['informacoes_pessoais']).add_prefix('informacoes_pessoais_')
informacoes_profissionais   = pd.json_normalize(df_apps_raw['informacoes_profissionais']).add_prefix('informacoes_profissionais_')
formacao_e_idiomas          = pd.json_normalize(df_apps_raw['formacao_e_idiomas']).add_prefix('formacao_e_idiomas_')
cargo_atual                 = pd.json_normalize(df_apps_raw['cargo_atual']).add_prefix('cargo_atual_')
cv_pt = df_apps_raw['cv_pt'].rename('cv_pt')
cv_en = df_apps_raw['cv_en'].rename('cv_en')

df_applicants = pd.concat([
    infos_basicas,
    informacoes_pessoais,
    informacoes_profissionais,
    formacao_e_idiomas,
    cargo_atual,
    cv_pt,
    cv_en
], axis=1)


In [42]:
# ---------------------- VAGAS ----------------------

df_vagas_raw = pd.read_json(path_vagas, orient='index')
df_vagas_raw = df_vagas_raw.reset_index().rename(columns={'index': 'id_vaga'})

df_info_basicas = pd.json_normalize(df_vagas_raw['informacoes_basicas']).add_prefix('info_')
df_perfil_vaga  = pd.json_normalize(df_vagas_raw['perfil_vaga']).add_prefix('perfil_')
df_beneficios   = pd.json_normalize(df_vagas_raw['beneficios']).add_prefix('benef_')

df_vagas = pd.concat([
    df_vagas_raw[['id_vaga']],
    df_info_basicas,
    df_perfil_vaga,
    df_beneficios
], axis=1)


In [43]:
# ---------------------- PROSPECTS ----------------------

df_prospects_raw = pd.read_json(path_prospects, orient='index')
df_prospects = df_prospects_raw.reset_index().rename(columns={'index': 'id_vaga'})
df_prospects_exploded = df_prospects.explode('prospects')

prospects_normalized = pd.json_normalize(df_prospects_exploded['prospects']).add_prefix('prospect_')

df_prospects_final = pd.concat([
    df_prospects_exploded.drop(columns=['prospects']).reset_index(drop=True),
    prospects_normalized
], axis=1)

In [44]:
# ---------------------- VISUALIZAÇÃO (opcional) ----------------------

print("\n🔍 Últimos 3 registros de cada DataFrame:")
print("Applicants:")
print(df_applicants.tail(3))
print("\nVagas:")
print(df_vagas.tail(3))
print("\nProspects:")
print(df_prospects_final.tail(3))


🔍 Últimos 3 registros de cada DataFrame:
Applicants:
      infos_basicas_telefone_recado infos_basicas_telefone  \
43058                           NaN                    NaN   
43059                           NaN                    NaN   
43060                           NaN                    NaN   

      infos_basicas_objetivo_profissional infos_basicas_data_criacao  \
43058                                 NaN                        NaN   
43059                                 NaN                        NaN   
43060                                 NaN                        NaN   

      infos_basicas_inserido_por infos_basicas_email infos_basicas_local  \
43058                        NaN                 NaN                 NaN   
43059                        NaN                 NaN                 NaN   
43060                        NaN                 NaN                 NaN   

      infos_basicas_sabendo_de_nos_por infos_basicas_data_atualizacao  \
43058                         

In [45]:
# Exportação
df_applicants.to_json(f'{destino}/applicants.json', orient='records', indent=4, force_ascii=False)
df_applicants.to_csv(f'{destino}/applicants.csv', index=False, encoding='utf-8-sig')

# Exportação
df_vagas.to_json(f'{destino}/vagas.json', orient='records', indent=4, force_ascii=False)
df_vagas.to_csv(f'{destino}/vagas.csv', index=False, encoding='utf-8-sig')

# Exportação
df_prospects_final.to_json(f'{destino}/prospects.json', orient='records', indent=4, force_ascii=False)
df_prospects_final.to_csv(f'{destino}/prospects.csv', index=False, encoding='utf-8-sig')